In [1]:
# Integration of biopsies and organ donors from 2 studies
import anndata, numpy as np, pandas as pd, scanpy as sc, imp, re, rpy2
%load_ext rpy2.ipython
sc.logging.print_versions()
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_rows', 1000)
import lpy # local lpy.py file

In [2]:
# Set some constants
adata = anndata.read_h5ad("int2.h5ad")
obskey_filteredcells= "filtered_cells"
adata.obs[ "sample_names" ].value_counts()

T3_C-0           19557
I-O-7-N_C-0      19043
I-O-5_8_C-0      16419
I-O-5_8-N_C-0    15453
I-O-6_9-N_C-0    15293
I-O-7_C-0        12892
I-O-6_9_C-0      12682
T2_C-0           12342
MH8919333        11970
S-HC015          10771
T1_C-0           10685
S-HC013          10477
S-HC017          10065
S-M067            9952
S-HC014           8393
S-S065            8129
MH8919332         7850
MH8919178         6948
newcastle74       6850
BGCV01_CV0904     6847
MH8919282         6802
MH8919283         6642
S-S054            5763
S-M044-1          5700
S-M044-2          5684
newcastle65       5604
S-S067            5592
S-S068            5298
S-HC010           5138
S-S055            5103
S-M051            5096
S-M052            4987
S-S057            4544
S-M046            4235
MH8919179         4214
S-M047            4172
S-M055            4117
S-M050            3821
S-S059            3772
S-HC008           3673
BGCV13_CV0934     3408
MH8919176         3085
BGCV14_CV0940     2957
BGCV09_CV09

In [3]:
adata.obs["Stage"] = lpy.applyFactorRename(adata.obs["Stage"], {"baseline":"baseline","progression":"progression","convalescence":"convalescence"}) # reorder cats

In [4]:
# imported annotation from celltypes from external dataset used for broad classification
target = "broad_integrated"
renamedico = {
 tuple(['7', '4','0', '5','16','1', '22','11', '13','19', '14','12', '6', '2', '24', '33']): 'Tcell',
 tuple(['8', '18', '32']): 'Bcell', 
 tuple(['15', '34','23', '20', '27','21','3','17','9','28', '25', '31', '30', '35']): 'Monocytes',    
 tuple(['26', '29','10', '27','36']): 'HighMT and doublets',
 'filtered': 'filtered'}
adata.obs[target] = lpy.applyFactorRename(adata.obs["leidenres2_sampl_everything"],renamedico)
tmp = lpy.mydoublerainbow([1,1,1,1])
adata.uns[target + "_colors"] = np.array(tmp, dtype = "object")
print(len(adata.obs[target].cat.categories))
print(adata.uns[target + "_colors"].shape)
adata.obs[target].value_counts()

5
(4,)


Tcell                  191587
Monocytes               78910
filtered                61774
Bcell                   22466
HighMT and doublets     17983
Name: broad_integrated, dtype: int64

In [5]:
mmm = pd.read_csv("azimuth_pred.tsv",delimiter='\t')
mmm

,cell,predicted.celltype.l2,predicted.celltype.l2.score,mapping.score
0,0,CD4_TCM,0.560974,0.753315
1,4,CD4_TEM,0.269865,0.397742
2,8,CD8_TEM,0.798537,0.527431
3,12,CD4_TCM,0.462367,0.705777
4,16,CD4_TCM,0.530068,0.212896
...,...,...,...,...
59583,238332,NK,0.993755,0.560735
59584,238336,CD14_Mono,0.997235,0.733066
59585,238340,NK,0.610048,0.569494
59586,238344,CD8_TEM,1.000000,0.850953


In [6]:
dadico = {adata.obs_names[x] : x  for x in range(372720)}
for i in range(59588):
    vaout3[dadico[tmp[i]]] = mmm["predicted.celltype.l2"][i]

In [7]:
vaout3[vaout3 == 0] = "NA"
mm2 = pd.Categorical(vaout3)
mm2.value_counts()

ASDC                     10
B_intermediate         1223
B_memory                896
B_naive                2767
CD14_Mono             12166
CD16_Mono              2106
CD4_CTL                 544
CD4_Naive              1934
CD4_Proliferating        49
CD4_TCM               12095
CD4_TEM                 646
CD8_Naive              1673
CD8_Proliferating        45
CD8_TCM                1039
CD8_TEM                8247
Eryth                    18
HSPC                     44
ILC                     124
MAIT                   1052
NA                   313132
NK                     8438
NK_CD56bright           442
NK_Proliferating        214
Plasmablast             267
Platelet                423
Treg                    373
cDC1                     60
cDC2                    906
dnT                      75
gdT                    1471
pDC                     241
dtype: int64

In [8]:
adata.obs["Azimuth_Hao"] = mm2

In [9]:
vaout3 = np.zeros(len(adata.obs["neigh"].cat.codes), dtype="object")
dal = ["S-M044-1","S-M046","S-M047","S-M067", "S-S054","S-S057","S-S061","S-S055","S-S059","S-M042-1","S-M042-2","S-M043-1","S-M043-2","S-M044-1","S-M044-2"]
vaout3[[  x in dal for x in adata.obs["sample_names"]] ] = "Mild"

dal = ["S-M044-2","S-M050","S-M051","S-M052","S-M055","S-S062","S-S064","S-S065","S-S067","S-S068","S-S035-1","S-S035-2","S-S035-3","S-S035-4","S-S036-1","S-S036-2","S-S036-3"]
vaout3[[  x in dal for x in adata.obs["sample_names"] ]] = "Seevere"

vaout3[ adata.obs["StudyName"]  =="Javi"] = "Mild"
vaout3[vaout3 == 0] = "Control"
vaout3[adata.obs["Stage"] == "baseline"] = "Control"


In [10]:
mm2 = pd.Categorical(vaout3)
mm2.value_counts()

Control    189297
Mild       135236
Seevere     48187
dtype: int64

In [11]:
# hani
# javi
# china 1-4

target = "sampl"
renamedico = {
 tuple(['BGCV01_CV0902', 'BGCV01_CV0904', 'BGCV02_CV0902', 'BGCV04_CV0911',
       'BGCV05_CV0929', 'BGCV08_CV0915', 'BGCV09_CV0917', 'BGCV10_CV0939',
       'BGCV12_CV0926', 'BGCV13_CV0934', 'BGCV14_CV0940', 'BGCV15_CV0944','MH8919176', 'MH8919177',
       'MH8919178', 'MH8919179', 'MH8919226', 'MH8919227', 'MH8919282',
       'MH8919283', 'MH8919332', 'MH8919333','doublet', 'newcastle65', 'newcastle74', 'unassigned','S-HC008','S-HC010', 'S-HC013',
       'S-HC014', 'S-HC015', 'S-HC017']) : "Control",
 tuple(['CVID1', 'CVID2', 'CVID3', 'CVID4', 'CVID5','S-M044', 'S-M046', 'S-M047', 'S-M050',
       'S-M051', 'S-M052', 'S-M055', 'S-M067']) : "Mild",
 tuple([ 'S-S054', 'S-S055', 'S-S057',
       'S-S059', 'S-S061', 'S-S062', 'S-S064', 'S-S065', 'S-S067', 'S-S068']) : "Severe"}

adata.obs["CovidSeverity"] = lpy.applyFactorRename(adata.obs["demultiplexed"],renamedico)
adata.obs["CovidSeverity"][adata.obs["Stage"] == "baseline"] = "Control"
lpy.RTable(adata.obs, "demultiplexed", )

,Control,Mild,Severe
CVID1,13624,46907,0
CVID2,9980,17413,0
CVID4,5160,9313,0
doublet,12518,0,0
MH8919333,11970,0,0
CVID3,2467,9098,0
S-M044,0,11384,0
S-HC015,10771,0,0
S-HC013,10477,0,0
S-HC017,10065,0,0


In [12]:
# Exports counts to R

spx = rpy2.robjects.FloatVector(adata.raw.X.data)
spp = rpy2.robjects.IntVector(adata.raw.X.indptr)
spi = rpy2.robjects.IntVector(adata.raw.X.indices)
spo = adata.obs
spv = adata.raw.var

In [13]:
%%R -i spx -i spp -i spi -i spo -i spv
# Make a Singlecell
library(SingleCellExperiment)
library(Matrix)
fun <- Matrix(0, 33759, length(spp) -1, sparse=T)
fun@x = spx
fun@i = spi
fun@p = spp
rownames(fun) = rownames(spv)
colnames(fun) = rownames(spo)

sce = SingleCellExperiment(assays = fun, rowData = spv, colData= spo)
saveRDS(sce , "/lustre/scratch117/cellgen/team292/lh20/covid_javi_all.sce.rds")
print(sce)

R[write to console]: Loading required package: SummarizedExperiment

R[write to console]: Loading required package: MatrixGenerics

R[write to console]: Loading required package: matrixStats

R[write to console]: 
Attaching package: ‘MatrixGenerics’


R[write to console]: The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiff

class: SingleCellExperiment 
dim: 33759 372720 
metadata(0):
assays(1): ''
rownames(33759): MIR1302-2HG FAM138A ... AB_CD258(LIGHT) AB_DR3(TRAMP)
rowData names(3): gene_ids feature_types n_cells
colnames(372720): T1_C-0_AAACCTGAGATAGCAT-1 T1_C-0_AAACCTGAGCTTATCG-1
  ... S-S068_TTTGTCAGTGATGTGG-43 S-S068_TTTGTCAGTTTAGCTG-43
colData names(110): souporcell demultiplexed ... TR_D_Genes TR_C_Genes
reducedDimNames(0):
altExpNames(0):


In [13]:
adata.write_h5ad("int3.h5ad")